### Run AutoQuake
#### Directory structure
##### SAC data (sac_parent_dir / date / *.SAC)
```
SAC/ -> sac_parent_dir     
└── 20240401/ -> date      
    ├── TW.station.00.HHZ.D.2024,093,20:22:12.SAC     
    └── TW.station.00.HHZ.D.2024,093,22:35:17.SAC            
```
##### HDF5 (h5_parent_dir / date / time_window.h5)
```
Hualien_0403_MiDAS/ -> h5_parent_dir     
└── 20240401/ -> date       
    ├── MiDAS_20240403_80700_81000.h5 -> time_window.h5     
    └── MiDAS_20240403_8100_8400.h5 
```
##### PZ file (pz_dir / *.pz)
```
pz_dir/ -> pz_dir     
├── SAC_PZs_TW_ZUZH_HH1_00_2019.001.00.00.00.0000_2599.365.23.59.59.99999
└── SAC_PZs_TW_ZUZH_HNZ_10_2019.001.00.00.00.0000_2599.365.23.59.59.99999
```
Date would acquire througth the reading of the gamma_events.csv from GaMMA.  
The reason why we need date is to automatically find the sac data due to we might have the events across the day.

### Scenarios (Seiemomter only, ~~DAS only, Both~~)
#### Seismometer only


1. Define the **Path** and some important parameters for later use.

In [ ]:
from pathlib import Path

result_path = Path('/home/patrick/Work/AutoQuake/testset_1029')
sac_parent_dir = Path('/raid1/share/for_patrick/AutoQuake_testset/SAC') # this is the testset, u can try it.
startdate = '20240401'
enddate = '20240403'
station = Path('/home/patrick/Work/EQNet/tests/hualien_0403/station_seis.csv')
pz_dir = Path('/raid1/share/for_patrick/AutoQuake_testset/PZ_dir/')
model_3d = Path('/home/patrick/Work/AutoQuake/H3DD/tomops_H14')
gamma_vel_model = Path('/home/patrick/Work/AutoQuake_pamicoding/GaMMA/Hualien_data_20240402/Hualien_1D.vel')

2. Create the object for each step.      
-  PhaseNet, GaMMA, H3DD     
       
Please check the detailed argument in the class if u want further manipulation, or you can just put your mouse on the PhaseNet object to see its docstring supported by VScode.

In [ ]:
from autoquake.picker import PhaseNet
from autoquake.associator import GaMMA
from autoquake.relocator import H3DD

phasenet = PhaseNet(
    data_parent_dir=sac_parent_dir,
    start_ymd=startdate,
    end_ymd=enddate,
    result_path=result_path,
    format='SAC',
    model='phasenet',
    device='cpu',
)

gamma = GaMMA(
    station=station,
    result_path=result_path,
    pickings=phasenet.picks,
    vel_model=gamma_vel_model,
)

h3dd = H3DD(
    gamma_event=gamma.events,
    gamma_picks=gamma.picks,
    station=gamma.station,
    model_3d=model_3d,
    event_name=result_path.name,
)

3. Run it through default scenario: AutoQuake       
> **⚠️ Important:** Since the important parameter setting only on **PhaseNet**, **GaMMA**, and **H3DD**, you can just put the object of these three steps into the run_autoquake function, and providing a **PZ_dir** contains the pz file. The **magnitude**, **polarity**, also **GAfocal** would use the result from these steps.

In [ ]:
from autoquake.scenarios import run_autoquake
run_autoquake(
    picker=phasenet,
    associator=gamma,
    relocator=h3dd,
    pz_dir=pz_dir,
    use_polarity=True, # default as True
    use_magnitude=True, # default as True
    use_focal=True # default as True
)

> **⚠️ Important:**  Once you understand the flow, please use python script to run it, jupyter notebook might crash due to the high memory loading.